In [1]:
import numpy as np
from qiskit import QuantumCircuit

import quimb
from qiskit_quimb import quimb_circuit
from collections import Counter
import torch

# Load the circuit

In [2]:
circuit = QuantumCircuit.from_qasm_file("P4_golden_mountain.qasm")

In [3]:
circuit.count_ops(), circuit.num_qubits

(OrderedDict([('u3', 10240), ('cz', 5096)]), 48)

# Run MPS simulation

Here we do a very low bond dimension simulation (relative to the number of qubits).

The simulation error may be virtually 1, but that is OK.

In [4]:
def to_backend(x):
    return torch.tensor(x, dtype=torch.complex64, device="cuda")

In [5]:
quimb_circuit = quimb_circuit(
    circuit, 
    quimb_circuit_class=quimb.tensor.CircuitPermMPS,  # This will help with long range gates 
    to_backend=to_backend,  # This will make it run faster (on GPU) but is not required
    max_bond=128,
    cutoff=1e-12,
    progbar=True,
)

max_bond=128, error~=1: 100%|##########| 15336/15336 [06:18<00:00, 40.52it/s]       


#### Undo the qubit permutation

In [6]:
qubit_mapping = list(range(quimb_circuit.N))
qubit_mapping = [quimb_circuit.qubits.index(q) for q in range(quimb_circuit.N)]
qubit_mapping = [qubit_mapping[q] for q in qubit_mapping]

# Generate samples

We don't need to take a lot of samples.

In [7]:
samples = [''.join(bs[q] for q in qubit_mapping) for bs in quimb_circuit.sample(1000, seed=1234)]
csamples = Counter(samples)

In [8]:
list(csamples.most_common())[:10]

[('110000111011100100100110011011011001000001101000', 1),
 ('011111111101000111010111100100100000000001111110', 1),
 ('010001001010110001100010011111111010111101111101', 1),
 ('001111100100000101011000011101111011011100110000', 1),
 ('100101101111010011100111000110000100100110001100', 1),
 ('100110000100001101100110101100010010100001111001', 1),
 ('100000000110011000000001001110110100101111110001', 1),
 ('000001101100001010111000011100011110100010110010', 1),
 ('111111100111101000000110100001001011100000101010', 1),
 ('110001010001100011100110101001111110110111111011', 1)]

# Distill peak bitstring

Distillation can be done by methods ranging from sophisticated clustering to just majority voting.

For these circuits I found it was enough to just do voting.

The bit probabilities can tell how "confident" we are in the bitstring. If they are all close to 0.5, try increasing the bond dimension.

In [9]:
bit_probs = np.array([[int(s) for s in ss] for ss in samples]).mean(axis=0)
bit_probs  # These are the individual bit probabilities

array([0.45 , 0.604, 0.521, 0.584, 0.563, 0.494, 0.381, 0.354, 0.365,
       0.569, 0.456, 0.404, 0.42 , 0.411, 0.611, 0.37 , 0.337, 0.427,
       0.45 , 0.3  , 0.217, 0.633, 0.562, 0.357, 0.514, 0.425, 0.579,
       0.631, 0.578, 0.538, 0.548, 0.589, 0.446, 0.382, 0.626, 0.43 ,
       0.593, 0.363, 0.395, 0.417, 0.306, 0.596, 0.595, 0.576, 0.562,
       0.546, 0.527, 0.485])

In [ ]:
voted_bitstring = "".join(str(i) for i in (bit_probs > 0.5).astype(int).tolist())
voted_bitstring  # Run this to see the answer!

Note that the output bitstring may not even be in the samples:

In [11]:
voted_bitstring in csamples

False